In [18]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.utils.data import Dataset
import plotly.express as px

In [19]:
#generate random data and study the correctness of the classifiers and other aspects.
X = np.random.normal(0,1,(1000,4))
X[:,2] = X[:,2]*2
X[:,3] = X[:,3]*1.5
X[:,1] += X[:,2] +X[:,3]
W = np.random.rand(4,1)
z = 1/(1 + np.exp(-( X @ W )))
y = (z >0.5).astype(int).reshape(-1)
X,y = torch.tensor(X),torch.tensor(y)

train_size = int(0.6 * len(X))
val_size = int(0.2 * len(X))
test_size = len(X) - train_size - val_size

class MyDataset(Dataset):
    def __init__(self,X,y):
        self.X = X
        self.y = y
        
    def __getitem__(self, index):
        x, y = self.X[index],self.y[index]
        return x, y
        
    def __len__(self):
        return len(self.X)
    
train_dataset, valid_dataset,test_dataset = torch.utils.data.random_split(MyDataset(X,y), [train_size, val_size, test_size])


In [88]:
from models.NonParametric import ParzenWindow
from Metrics import ComputeMetrics
compute_metrics = ComputeMetrics(2)
metrics_dict = {}
hyperparamter_grid_params = []
for whitten in [False,True]:
    for h in tqdm(np.logspace(-3, 1, 20)):
        X_train,y_train = train_dataset[:]
        pw = ParzenWindow(h=h,whitten=whitten)
        pw.fit(X_train,y_train)
        #valid
        X_val,y_val = valid_dataset[:]
        y_pred = pw.predict(X_val)
        metrics_dict[h] = compute_metrics(y_pred,y_val)

    #choice for hyper parameters
    hyperparamter_grid = []
    for h,metrics in metrics_dict.items():
        hyperparamter_grid.append([h,metrics[1]["f1"]])
    hyperparamter_grid_params.append(np.array(hyperparamter_grid))


100%|██████████| 20/20 [00:00<00:00, 96.21it/s] 


In [89]:
import plotly.graph_objects as go
fig = go.Figure(
    data=[go.Scatter(x=hyperparamter_grid_params[0][:,0], y=hyperparamter_grid_params[0][:,1],name="non - whitened data")],
    layout=go.Layout(
        xaxis_title="h", yaxis_title="f1 score", width=600,height=400,title='f1 score with varying window width (h)'
    )
)
fig.add_trace( go.Scatter(x=hyperparamter_grid_params[1][:,0], y=hyperparamter_grid_params[1][:,1],name = "whitened data"))
fig.show()

In [114]:
for i,whitten in enumerate([False,True]):
   best_hyperparm = hyperparamter_grid_params[i][hyperparamter_grid_params[i][:,1].argmax()][0]
   pw = ParzenWindow(h=best_hyperparm,whitten=whitten)
   pw.fit(X_train,y_train)
   #compute model variance
   metrics_dict = {}
   for i in range(10):
      test_set_size = int(len(test_dataset) * 0.8)
      leftout_size = len(test_dataset) - test_set_size
      train_set, valid_set = torch.utils.data.random_split(test_dataset, [test_set_size, leftout_size])
      y_pred = pw.predict(X_val)
      metrics_dict[i] = compute_metrics(y_pred,y_val)
   #choice for hyper parameters
   epoch_grid = []
   for h,metrics in metrics_dict.items():
      epoch_grid.append(metrics[1]["f1"].item())
   print(np.mean(epoch_grid), np.var(epoch_grid))

0.9699879884719849 0.0
1.0 0.0


In [220]:
DATA_DIR = "colored_mnist"
TRAIN_BATCH_SIZE = 30000
TEST_BATCH_SIZE = 1000
LOG  = "disabled" #dryrun #online
import os,wandb
from Dataloader import WholeDataset
from torch.utils.data import DataLoader
os.environ["WANDB_API_KEY"]= "13978bc398bdedd79f4db560bfb4b79e2db711b5"
wandb.login()
wandb.init(
    mode=LOG,
    project="Biased MNIST",
    config={
        "epochs": 100,
        "batch_size": 1000,
        "lr": 1e-2,
        "adversery_weight":False,
        "permutation":False
    })


class Args(object):
    def __init__(self,data_split):
        self.data_dir = DATA_DIR
        self.data_split = data_split
        self.color_var = 0.040
config = wandb.config

args_train = Args("train")
train_dataset = WholeDataset(args_train)
args_test  = Args("test")
test_dataset = WholeDataset(args_test)

train_dataloader = DataLoader(train_dataset,shuffle=True,batch_size=300000,num_workers=4,pin_memory=True,persistent_workers=True)
test_dataloader = DataLoader(test_dataset,shuffle=False,batch_size=5000,num_workers=4,pin_memory=True,persistent_workers=True)


X_t,l_t,y_t,s_t  = next(iter(train_dataloader))
X_v,l_v,y_v,s_v  = next(iter(test_dataloader))

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.
/research/hal-gaudisac/project802/Dataloader.py:104: UserWarning:

The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at /opt/conda/conda-bld/pytorch_1674893616450/work/torch/csrc/utils/tensor_numpy.cpp:206.)

/research/hal-gaudisac/project802/Dataloader.py:104: UserWarning:

The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered in

In [221]:
X_embedded = TSNE(n_components=2, learning_rate='auto',init='random', perplexity=3).fit_transform(X_t.flatten(start_dim=1))

In [ ]:

fig = px.scatter(
    X_embedded, x=0, y=1,
    color=y_t
)
fig.show()